<a href="https://colab.research.google.com/github/wozzin/Natural_Language_Processing/blob/main/B_build_dtm_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install & import the libraries needed
!pip3 install pandas
!pip3 install scikit-learn
from typing import List
from math import log
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
# for printing out all the columns of a pandas dataframe https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf
pd.set_option('display.max_columns', None)

In [2]:
# The corpus
CORPUS = [
    'this is the first document',
    'the first document is this',
    'this is the second document',
    'and this is the third document',
    'is this the first document'
]


In [3]:
# we will reuse the functions we completed in the previous tutorial
def tf(term: str, doc: str) -> int:
    # typing을 하는 이유: 오류를 미연에 방지하기 위해서 (단, IDE를 사용하는 경우에만)
    # 더 정확한 버전 (is가 this에 포함 문제가 되므로)
    tf = sum([
      word == term     
      for word in doc.split(" ")  # word count
    ])
    return tf


def build_dtm(corpus: List[str]) -> pd.DataFrame:
    """
    we reuse what we implemented in previous tutorial
    :param corpus:
    :return:
    """
    # build vocabulary - use nested list comprehension, set, and list
    words = [
        word
        for doc in corpus
        for word in doc.split(" ")
    ]
    vocab = list(set(words))

    # populate dtm, ith a nested for loop
    dtm = [
           [tf(term, doc) for term in vocab]  # row
           for doc in corpus
    ]

    # return dtm as a pandas dataframe
    dtm = pd.DataFrame(data=dtm, columns=vocab)
    return dtm



In [4]:
# complete this function
def idf(term: str, corpus: List[str]) -> float:
    ### TODO 1 ###
    # idf = log(n / (1 + df))
    n = len(corpus)
    # df: term이 출현한 문서의 빈도수
    # 김영학09:04 split(" ")을 사용 안 하면 문제가 생길까요?
    # 넵, e.g. is ,this
    df = sum([
          term in doc.split(" ")  # T, F
          for doc in corpus
    ])
    idf = log(n / (1 + df))
    ##############
    return idf

In [5]:
print(idf("this", CORPUS))
print(idf("eubin", CORPUS))

-0.1823215567939546
1.6094379124341003


In [6]:
def build_dtm_with_tfidf(corpus: List[str]) -> pd.DataFrame:
    # access the columns with df.columns
    # multiply idfs to each row with numpy's broadcast mul
    dtm = build_dtm(corpus) # tf
    ### TODO 2 ####
    # reuse dtm to build dtm_tfidf
    # use dtm.columns, idf, np.array(), dtm.to_numpy() and broadcast multiplication
    idfs: List[float] = [
                         idf(term, corpus)
                         for term in dtm.columns
    ]
    dtm_tfidf: np.array = dtm.to_numpy() * np.array(idfs)  # 리스트, 혹은 넘파이 배열
    ############### 
    return pd.DataFrame(data=dtm_tfidf, columns=dtm.columns)


In [7]:
# build dtm, and one with  
dtm = build_dtm(CORPUS)
dtm_tfidf = build_dtm_with_tfidf(CORPUS)
print(dtm)
print(dtm_tfidf)

   the  first  third  this  document  second  and  is
0    1      1      0     1         1       0    0   1
1    1      1      0     1         1       0    0   1
2    1      0      0     1         1       1    0   1
3    1      0      1     1         1       0    1   1
4    1      1      0     1         1       0    0   1
        the     first     third      this  document    second       and  \
0 -0.182322  0.223144  0.000000 -0.182322 -0.182322  0.000000  0.000000   
1 -0.182322  0.223144  0.000000 -0.182322 -0.182322  0.000000  0.000000   
2 -0.182322  0.000000  0.000000 -0.182322 -0.182322  0.916291  0.000000   
3 -0.182322  0.000000  0.916291 -0.182322 -0.182322  0.000000  0.916291   
4 -0.182322  0.223144  0.000000 -0.182322 -0.182322  0.000000  0.000000   

         is  
0 -0.182322  
1 -0.182322  
2 -0.182322  
3 -0.182322  
4 -0.182322  


다음과 같은 결과가 나와야 합니다 (단어의 순서는 달라도 괜찮습니다):
```
   the  and  third  this  is  first  document  second
0    1    0      0     1   2      1         1       0
1    1    0      0     1   2      1         1       0
2    1    0      0     1   2      0         1       1
3    1    1      1     1   2      0         1       0
4    1    0      0     1   2      1         1       0
        the       and     third      this        is     first  document  \
0 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   
1 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   
2 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.000000 -0.182322   
3 -0.182322  0.916291  0.916291 -0.182322 -0.364643  0.000000 -0.182322   
4 -0.182322  0.000000  0.000000 -0.182322 -0.364643  0.223144 -0.182322   

     second  
0  0.000000  
1  0.000000  
2  0.916291  
3  0.000000  
4  0.000000  

```

In [8]:
# compare the two
print(cosine_similarity(dtm.to_numpy(), dtm.to_numpy()))
print(cosine_similarity(dtm_tfidf.to_numpy(), dtm_tfidf.to_numpy()))

[[1.         1.         0.8        0.73029674 1.        ]
 [1.         1.         0.8        0.73029674 1.        ]
 [0.8        0.8        1.         0.73029674 0.8       ]
 [0.73029674 0.73029674 0.73029674 1.         0.73029674]
 [1.         1.         0.8        0.73029674 1.        ]]
[[1.         1.         0.31538537 0.23104796 1.        ]
 [1.         1.         0.31538537 0.23104796 1.        ]
 [0.31538537 0.31538537 1.         0.10015744 0.31538537]
 [0.23104796 0.23104796 0.10015744 1.         0.23104796]
 [1.         1.         0.31538537 0.23104796 1.        ]]


다음과 같은 결과가 나와야 합니다:
```
[[1.         1.         0.875      0.82495791 1.        ]
 [1.         1.         0.875      0.82495791 1.        ]
 [0.875      0.875      1.         0.82495791 0.875     ]
 [0.82495791 0.82495791 0.82495791 1.         0.82495791]
 [1.         1.         0.875      0.82495791 1.        ]]
[[1.         1.         0.4227912  0.31662902 1.        ]
 [1.         1.         0.4227912  0.31662902 1.        ]
 [0.4227912  0.4227912  1.         0.16251445 0.4227912 ]
 [0.31662902 0.31662902 0.16251445 1.         0.31662902]
 [1.         1.         0.4227912  0.31662902 1.        ]]

```

## 다음의 질문에 답해주세요!

> `dtm`으로 구한 유사도 행렬 대비, `dtm_tfidf`로 구한 유사도 행렬은 어떤 점이 개선되었나요? 그 이유는?

- 의미 구분을 더 잘하게 되었다. 김영학09:11 - hit the nail on the head!
의미가 같지 않은 문장끼리의 유사성이 1에서 멀어짐
유사성이 1에서 멀어진 이유는 흔하게 나타나는(빈도수가 높은) 단어에 대해 가중치를 낮추어 주어서 반영하였기 때문


> `dtm_tfidf`로도 해결할수 없는 문제를 발견할 수 있나요?
- 카운트 기반 -> 순서를 고려 x (bag-of-words) / 순서를 고려하는 모델 (e.g. RNN, transformer, BERT)
- 1. 순서를 고려하지 않아서 this is the first document의 평서문 문장과 비록 ?가 없지만 어순상 is this the first document의 의문문에 대한 유사성 1로 나타나 똑같은 의미를 갖는 문장으로 판단함
-  distributional semantics: 의미 <- 데이터, 데이터에 문제가 있으면 의미에도 문제.
실제로 중요하지 않은 단어 'and'의 df가 1로 흔하지 않은(빈도수가 낮은) 단어로 판단함. 즉, 가중치가 높은 중요한 단어로 고려됨. 주어진 데이터에 따라 가중치가 결정됨.

